In [8]:
import pandas as pd

import plotly.express as px

from main import GameResult

In [ ]:
def _sanity_check(x, CAT, animal_cols):
    cat_finished = x[CAT] == GameResult.FINISHED.name
    any_animal_finished = any(x[col] == GameResult.FINISHED.name for col in animal_cols)
    all_animals_chased = all(x[col] == GameResult.CHASED.name for col in animal_cols)
    assert cat_finished != any_animal_finished
    if any_animal_finished or all_animals_chased:
        assert any_animal_finished != all_animals_chased


def get_winner(x):
    CAT = "C"
    animal_cols = ["B", "S", "M"]
    _sanity_check(x, CAT, animal_cols)
    result = None
    for col in [CAT] + animal_cols:
        if x[col] == GameResult.FINISHED.name:
            if result is not None:
                result = col
            else:
                raise ValueError("W?")



    return result

In [19]:
df = pd.read_csv("results.csv")
display(df.describe())
display(df.groupby(by="strategy").describe())

,epoch,position,game_result,shortcut_position,game_id,timestamp
count,45720.000000,45720.000000,45720.000000,34290.000000,45720.000000,4.572000e+04
mean,19.895101,15.673710,-0.579768,21.000000,148.042170,1.746453e+12
std,13.464423,12.976511,6.134285,8.165085,85.722404,8.301741e+02
min,-1.000000,0.000000,-2.000000,11.000000,0.000000,1.746453e+12
25%,8.000000,6.000000,-2.000000,11.000000,74.000000,1.746453e+12
50%,19.000000,9.000000,-1.000000,21.000000,150.000000,1.746453e+12
75%,31.000000,24.000000,-1.000000,31.000000,223.000000,1.746453e+12
max,61.000000,50.000000,50.000000,31.000000,299.000000,1.746453e+12


epoch                                                    \
                     count       mean        std  min  25%   50%   75%   max   
strategy                                                                       
CLOSEST_RUN_AWAY   15716.0  19.910410  13.268288 -1.0  9.0  19.0  30.0  61.0   
ONLY_ONE_RUN_AWAY  14612.0  19.166165  13.126803 -1.0  8.0  18.0  30.0  56.0   
RANDOM_SINGLE      15392.0  20.571466  13.936585 -1.0  8.0  20.0  32.0  59.0   

                  position             ... game_id        timestamp  \
                     count       mean  ...     75%    max     count   
strategy                               ...                            
CLOSEST_RUN_AWAY   15716.0  15.852889  ...   176.0  199.0   15716.0   
ONLY_ONE_RUN_AWAY  14612.0  15.206200  ...   274.0  299.0   14612.0   
RANDOM_SINGLE      15392.0  15.934576  ...    74.0   99.0   15392.0   

                                                                         \
                           mean         std           min           25%   
strategy                                                                  
CLOSEST_RUN_AWAY   1.746453e+12  280.070938  1.746453e+12  1.746453e+12   
ONLY_ONE_RUN_AWAY  1.746453e+12  244.063630  1.746453e+12  1.746453e+12   
RANDOM_SINGLE      1.746453e+12  293.394296  1.746453e+12  1.746453e+12   

                                                             
                            50%           75%           max  
strategy                                                     
CLOSEST_RUN_AWAY   1.746453e+12  1.746453e+12  1.746453e+12  
ONLY_ONE_RUN_AWAY  1.746453e+12  1.746453e+12  1.746453e+12  
RANDOM_SINGLE      1.746453e+12  1.746453e+12  1.746453e+12  

[3 rows x 48 columns]

In [11]:
ds = []
for g_id in df.game_id.unique():
    df_game = df[df["game_id"] == g_id]
    last_epoch = df_game.epoch.max()
    df_last_epoch = df_game[df_game["epoch"] == last_epoch]
    d = {"end_epoch": last_epoch, "game_id": g_id, "strategy": df_last_epoch["strategy"].iloc[0]}
    for idx, row in df_last_epoch.iterrows():
        d[row["name"]] = GameResult(row["game_result"]).name
    ds.append(d)

df_game_result = pd.DataFrame(ds)
df_game_result["winner"] = df_game_result.apply(get_winner, axis=1)

In [17]:
df_game_result[df_game_result["C"] != GameResult.FINISHED.name]
px.histogram(df_game_result, "winner", color="strategy")


In [16]:
px.histogram(df_game_result, x="end_epoch", nbins=len(df_game_result.end_epoch.unique()), color="strategy")